In [1]:
# import
import zipfile


In [5]:
# extract zip file quickly and in gitignore'd folder

zip_file_path = "./data/archive.zip"

# Step 2: Extract the downloaded zip file
extracted_folder_path = "./data/RoadDetectionFiles"
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

In [6]:
# Define paths

test_image = './data/RoadDetectionFiles/Test/image'
train_image = './data/RoadDetectionFiles/Train/image'
validation_image = './data/RoadDetectionFiles/Validation/image'

test_centerline = './data/RoadDetectionFiles/Test/centerline'
train_centerline = './data/RoadDetectionFiles/Train/centerline'
validation_centerline = './data/RoadDetectionFiles/Validation/centerline'

test_label = './data/RoadDetectionFiles/Test/label'
train_label = './data/RoadDetectionFiles/Train/label'
validation_label = './data/RoadDetectionFiles/Validation/label'